# Préparation des data pour étudiants

Les étudiants ont des difficultés de récupérer les tweets. Ce notebook a pour but de préparer des données propres et avec toutes les infos utiles, ainsi qu'un échantillonnage de tweets pour effectuer l'étude scattertext.

In [5]:
import pandas as pd 
import pickle
import re

In [3]:
%cd ../../data

C:\Users\ManonRICHARD\Documents\PFE\twitter-fakenews\data


## Import des données

### Les sites decodex 

In [6]:
decodex_data = pd.read_csv("twitter_accounts_decodex.csv", sep=";")

decodex_data['path_website'] = decodex_data['path_website'].apply(lambda txt : re.sub('twitter.(com|fr)/', '', txt))
decodex_data['path_website'] = decodex_data['path_website'].apply(lambda txt : re.sub(r'\?lang=fr', '', txt))

decodex_data.head()

,unique_id,liability,description,name,namelow,liability_label,path_website,twitter
0,1982,2,France-Soir était un journal fondé par Pierre ...,France Soir,france-soir,site qui diffuse régulièrement de la fausse in...,france_soir,True
1,1981,2,"Thierry Casasnovas, vidéaste et animateur de l...",Régénère (Thierry Casasnovas),regenere-thierry-casasnovas,site qui diffuse régulièrement de la fausse in...,ThierryRegenere,True
2,1980,2,Silvano Trotta est le créateur d’une société s...,Silvano Trotta,silvano-trotta,site qui diffuse régulièrement de la fausse in...,silvano_trotta,True
3,1976,2,Cette propriétaire d’un magasin de chaussures ...,Ema Krusi,ema-krusi,site qui diffuse régulièrement de la fausse in...,emakrusi,True
4,1973,2,Salim Laïbi est un chirurgien-dentiste connu s...,Salim Laïbi (Le Libre Penseur),salim-laibi-le-libre-penseur,site qui diffuse régulièrement de la fausse in...,llp_le_vrai,True


### Les tweets

In [25]:
with open("tableauTweets.txt", "rb") as file :   
    tweets_all_accounts = pickle.load(file)

In [26]:
tweets_all_accounts.head()

,user_id,user_screen_name,id_tweet,created_at,tweet,retweet_count,favorite_count
0,78957336,france_soir,1466789582213681162,2021-12-03 15:21:02+00:00,🚨Le professeur Raoult écope d’un blâme de l’Or...,50,101
1,78957336,france_soir,1466768535921831946,2021-12-03 13:57:24+00:00,RT @DIVIZIO1: Recours déposé !,643,0
2,78957336,france_soir,1466757902773932035,2021-12-03 13:15:09+00:00,"🎙BEST OF - Pr Michel Drancourt : IHU, épidémie...",27,59
3,78957336,france_soir,1466739848509890566,2021-12-03 12:03:25+00:00,"📚 Jonathan Curiel, directeur général adjoint d...",14,37
4,78957336,france_soir,1466731706912751619,2021-12-03 11:31:04+00:00,🖍 Le dessin du jour\n&gt; à retrouver sur http...,39,79


In [27]:
tweets_all_accounts = tweets_all_accounts.drop_duplicates(["id_tweet"])
# je pense que des doublons ce sont créés à cause des rate limit reached, wait for ... 
# car l'algo semble reprendre la récupération du début 

In [28]:
tweets_all_accounts.shape

(1072363, 7)

### Nombre de followers par compte

In [14]:
df_followers_count = pd.read_csv("df_followers_count_by_user.csv", sep=";", encoding='UTF-8')

In [15]:
df_followers_count.head(3)

,screen_name,count_followers
0,france_soir,98322
1,ThierryRegenere,1967
2,emakrusi,46639


## Jointure

In [29]:
tweets_all_accounts = pd.merge(tweets_all_accounts, decodex_data, left_on="user_screen_name", right_on="path_website")
tweets_all_accounts = pd.merge(tweets_all_accounts, df_followers_count, left_on="user_screen_name", right_on="screen_name")

In [30]:
tweets_all_accounts = tweets_all_accounts.drop(["unique_id","path_website","twitter","screen_name","namelow","name"], axis=1)

In [31]:
tweets_all_accounts.head(3)

,user_id,user_screen_name,id_tweet,created_at,tweet,retweet_count,favorite_count,liability,description,liability_label,count_followers
0,78957336,france_soir,1466789582213681162,2021-12-03 15:21:02+00:00,🚨Le professeur Raoult écope d’un blâme de l’Or...,50,101,2,France-Soir était un journal fondé par Pierre ...,site qui diffuse régulièrement de la fausse in...,98322
1,78957336,france_soir,1466768535921831946,2021-12-03 13:57:24+00:00,RT @DIVIZIO1: Recours déposé !,643,0,2,France-Soir était un journal fondé par Pierre ...,site qui diffuse régulièrement de la fausse in...,98322
2,78957336,france_soir,1466757902773932035,2021-12-03 13:15:09+00:00,"🎙BEST OF - Pr Michel Drancourt : IHU, épidémie...",27,59,2,France-Soir était un journal fondé par Pierre ...,site qui diffuse régulièrement de la fausse in...,98322


In [32]:
tweets_all_accounts.id_tweet = tweets_all_accounts.id_tweet.astype('str')
tweets_all_accounts.id_tweet.describe()

count                  926188
unique                 926188
top       1466789582213681162
freq                        1
Name: id_tweet, dtype: object

### Export des données

In [34]:
with open("data_tweets.txt", "wb") as fp :   #Pickling
    pickle.dump(tweets_all_accounts, fp)

In [ ]:
with open("data_tweets.txt", "rb") as file :   
    data_tweets = pickle.load(file)

### Echantillon des données